# Лабораторная №3

Вначале импортируем нужные вещи:

In [1]:
from __future__ import print_function
import numpy
numpy.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.utils import np_utils

from scipy.misc import imresize

Using TensorFlow backend.


Ниже приведенный код обрабатывает тестовые картинки (приводя их размер к 64х64) и разбивает их на пакеты по 5000 картинок.

In [6]:
X_train = numpy.load('test.npy')
print(X_train.shape)
BUFF_SIZE = 5000
IMAGE_SIZE = (64, 64)
for i in range(0, len(X_train), BUFF_SIZE):
    tmp = numpy.zeros((min(BUFF_SIZE, len(X_train)-i), 64, 64))
    for j in range(i, min(len(X_train), i + BUFF_SIZE)):
        #print(i, j, X_train[j])
        tmp[j - i] = imresize(X_train[j], IMAGE_SIZE)
    tmp = numpy.dstack(tmp)
    tmp = numpy.rollaxis(tmp, 2)
    print(tmp.shape)
    #tmp = numpy.vstack((tmp, y_train[i:min(len(y_train), i+BUFF_SIZE)]))
    print(i // BUFF_SIZE)
    numpy.save(str(i // BUFF_SIZE) + '-normalized_64_test.npy', tmp)
del X_train

(41428,)
(5000, 64, 64)
0
(5000, 64, 64)
1
(5000, 64, 64)
2
(5000, 64, 64)
3
(5000, 64, 64)
4
(5000, 64, 64)
5
(5000, 64, 64)
6
(5000, 64, 64)
7
(1428, 64, 64)
8


В ячейках ниже я считываю тренировочный сет, преобразовываю к картинки к размеру 64х64 и разбиваю на пакеты. Для более удобного чтения (и записи, в принципе, тоже), ответы хранятся отдельно.

In [6]:
data = numpy.load('train.npy')
X_train = data[:, 0]
y_train = data[:, 1]

In [7]:
print(len(X_train[1]), len(X_train))

71 166708


In [8]:
BUFF_SIZE = 5000
IMAGE_SIZE = (64, 64)
for i in range(0, len(X_train), BUFF_SIZE):
    tmp = numpy.zeros((min(BUFF_SIZE, len(X_train)-i), 64, 64))
    for j in range(i, min(len(X_train), i + BUFF_SIZE)):
        #print(i, j, X_train[j])
        tmp[j - i] = imresize(X_train[j], IMAGE_SIZE)
    tmp = numpy.dstack(tmp)
    tmp = numpy.rollaxis(tmp, 2)
    print(tmp.shape, y_train.shape)
    #tmp = numpy.vstack((tmp, y_train[i:min(len(y_train), i+BUFF_SIZE)]))
    print(i // BUFF_SIZE)
    numpy.save(str(i // BUFF_SIZE) + '-normalized_64_pictures.npy', tmp)
    numpy.save(str(i // BUFF_SIZE) + '-normalized_64_answers.npy', y_train[i:min(len(y_train), i+BUFF_SIZE)])

(5000, 64, 64) (166708,)
0
(5000, 64, 64) (166708,)
1
(5000, 64, 64) (166708,)
2
(5000, 64, 64) (166708,)
3
(5000, 64, 64) (166708,)
4
(5000, 64, 64) (166708,)
5
(5000, 64, 64) (166708,)
6
(5000, 64, 64) (166708,)
7
(5000, 64, 64) (166708,)
8
(5000, 64, 64) (166708,)
9
(5000, 64, 64) (166708,)
10
(5000, 64, 64) (166708,)
11
(5000, 64, 64) (166708,)
12
(5000, 64, 64) (166708,)
13
(5000, 64, 64) (166708,)
14
(5000, 64, 64) (166708,)
15
(5000, 64, 64) (166708,)
16
(5000, 64, 64) (166708,)
17
(5000, 64, 64) (166708,)
18
(5000, 64, 64) (166708,)
19
(5000, 64, 64) (166708,)
20
(5000, 64, 64) (166708,)
21
(5000, 64, 64) (166708,)
22
(5000, 64, 64) (166708,)
23
(5000, 64, 64) (166708,)
24
(5000, 64, 64) (166708,)
25
(5000, 64, 64) (166708,)
26
(5000, 64, 64) (166708,)
27
(5000, 64, 64) (166708,)
28
(5000, 64, 64) (166708,)
29
(5000, 64, 64) (166708,)
30
(5000, 64, 64) (166708,)
31
(5000, 64, 64) (166708,)
32
(1708, 64, 64) (166708,)
33


Следующая ячейка является подготовительной: в ней можно удалить все загруженные ранее данные из памяти, также в ней происходит создание словаря: неудобному номеру иероглифа сопоставляется число от 0 до 499.

In [2]:
#del X_train
#del data
#del y_train
data = numpy.load('train.npy')
Y_train = data[:, 1]
class_values = dict()
for elem in Y_train:
    if not(elem in class_values):
        class_values[elem] = len(class_values)
del Y_train
del data

Одна из нейронок, которые я использовал (всего их было не меньше 5):

In [3]:
model = Sequential()
model.add(Convolution2D(24, 3, 3,
                        border_mode='valid',
                        input_shape=(64, 64, 1)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(16, 3, 3,
                        border_mode='same'))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(len(class_values)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer=Adadelta(),
            metrics=['categorical_accuracy'])

Ячейка для обучения:

In [4]:
for ep in range(8):
    for batch in range(0, 29):
        X_train = numpy.load(str(batch) + '-normalized_64_pictures.npy')
        Y_train = numpy.load(str(batch) + '-normalized_64_answers.npy')

        for i in range(len(Y_train)):
            Y_train[i] = class_values[Y_train[i]] 
        Y_train = np_utils.to_categorical(Y_train, len(class_values))

        #print(X_train.shape, Y_train.shape)
        X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
        X_train = X_train.astype('float32')
        X_train /= 255
    #X_train.shape = (X_train.shape[0], 32 * 32)
    #print(X_train.shape, len(X_train[0]), len(X_train[1]), X_train[0].shape)
    #for i in range(len(X_train)):
    #    X_train[i] = X_train[i].reshape((70, 70, 1))
    #print(X_train.shape, Y_train.shape)
    #X_train = numpy.zeros((10000, 70, 70, 1))
    #X_train.shape = (X_train.shape[0], 70, 70, 1)

        history = model.fit(X_train, Y_train,
                        batch_size=150, nb_epoch=1,
                        verbose=0, validation_data=(X_train, Y_train))
        score = model.evaluate(X_train, Y_train, verbose=0)
        print(ep + 1, batch, score)
        del X_train
        del Y_train
    print("LET'S TEST IT!")
    for batch in range(29, 34):
        X_train = numpy.load(str(batch) + '-normalized_64_pictures.npy')
        Y_train = numpy.load(str(batch) + '-normalized_64_answers.npy')

        for i in range(len(Y_train)):
            Y_train[i] = class_values[Y_train[i]] 
        Y_train = np_utils.to_categorical(Y_train, len(class_values))

        X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
        X_train = X_train.astype('float32')
        X_train /= 255

        #history = model.fit(X_train, Y_train,
        #                batch_size=128, nb_epoch=10,
        #                verbose=0, validation_data=(X_train, Y_train))
        score = model.evaluate(X_train, Y_train, verbose=0)
        print(batch, score)
        del X_train
        del Y_train

1 0 [6.2141512519836422, 0.0033999999999999998]
1 1 [6.2136362731933596, 0.0055999999999999999]
1 2 [6.2115693199157711, 0.0060000000000000001]
1 3 [6.206251860809326, 0.0066]
1 4 [6.1926560005187987, 0.0091999999999999998]
1 5 [6.1258970474243162, 0.019400000000000001]
1 6 [5.7830064231872562, 0.026800000000000001]
1 7 [4.8109211975097654, 0.058000000000000003]
1 8 [4.3903656204223633, 0.13200000000000001]
1 9 [3.7689528434753417, 0.1726]
1 10 [3.1505519550323484, 0.24660000000000001]
1 11 [2.6424408191680908, 0.41060000000000002]
1 12 [2.7631671539306639, 0.36080000000000001]
1 13 [1.9648718183517455, 0.52780000000000005]
1 14 [1.6914605613708495, 0.59379999999999999]
1 15 [1.6944112820625306, 0.60899999999999999]
1 16 [1.654318949699402, 0.61739999999999995]
1 17 [1.483735572052002, 0.69140000000000001]
1 18 [1.2300259890556335, 0.73360000000000003]
1 19 [1.4620216913223267, 0.67759999999999998]
1 20 [1.3886754035949707, 0.70979999999999999]
1 21 [1.2677845747947694, 0.7318000000000

Еще одна нейронка:

In [3]:
model = Sequential()
model.add(Convolution2D(28, 3, 3,
                        border_mode='valid',
                        input_shape=(64, 64, 1)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(len(class_values)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer=Adadelta(),
            metrics=['categorical_accuracy'])

Ячейка для дообучения:

In [16]:
for ep in range(1):
    for batch in range(33, -1, -1):
        X_train = numpy.load(str(batch) + '-normalized_64_pictures.npy')
        Y_train = numpy.load(str(batch) + '-normalized_64_answers.npy')

        for i in range(len(Y_train)):
            Y_train[i] = class_values[Y_train[i]] 
        Y_train = np_utils.to_categorical(Y_train, len(class_values))

        #print(X_train.shape, Y_train.shape)
        X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
        X_train = X_train.astype('float32')
        X_train /= 255
        history = model.fit(X_train, Y_train,
                        batch_size=150, nb_epoch=1,
                        verbose=0, validation_data=(X_train, Y_train))
        #score = model.evaluate(X_train, Y_train, verbose=0)
        print(ep + 1, batch)
        del X_train
        del Y_train
    print("LET'S TEST IT!")
    for batch in range(0, 34, 3):
        X_train = numpy.load(str(batch) + '-normalized_64_pictures.npy')
        Y_train = numpy.load(str(batch) + '-normalized_64_answers.npy')

        for i in range(len(Y_train)):
            Y_train[i] = class_values[Y_train[i]] 
        Y_train = np_utils.to_categorical(Y_train, len(class_values))

        X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
        X_train = X_train.astype('float32')
        X_train /= 255

        #history = model.fit(X_train, Y_train,
        #                batch_size=128, nb_epoch=10,
        #                verbose=0, validation_data=(X_train, Y_train))
        score = model.evaluate(X_train, Y_train, verbose=1)
        print(batch, score)
        del X_train
        del Y_train

1 33
1 32
1 31
1 30
1 29
1 28
1 27
1 26
1 25
1 24
1 23
1 22
1 21
1 20
1 19
1 18
1 17
1 16
1 15
1 14
1 13
1 12
1 11
1 10
1 9
1 8
1 7
1 6
1 5
1 4
1 3
1 2
1 1
1 0
LET'S TEST IT!
5000/5000 [==============================] - 30s    
0 [0.0069378992570796978, 0.999]
5000/5000 [==============================] - 29s    
3 [0.014543042958853766, 0.99780000000000002]
5000/5000 [==============================] - 28s    
6 [0.018294929161481559, 0.99560000000000004]
5000/5000 [==============================] - 30s    
9 [0.013953546775877476, 0.99739999999999995]
5000/5000 [==============================] - 31s    
12 [0.022625366052566095, 0.99539999999999995]
5000/5000 [==============================] - 29s    
15 [0.03118055875301361, 0.99260000000000004]
5000/5000 [==============================] - 32s    
18 [0.035995272591710092, 0.99219999999999997]
5000/5000 [==============================] - 29s    
21 [0.036756646072492002, 0.99160000000000004]
5000/5000 [==============================] 

Ячейка для записи предсказаний нейронки в файл:

In [17]:
output = open('pred_64_13.txt', 'w')
output.write("Id,Category\n")
c = 1
for batch in range(0, 9):
    X_test = numpy.load(str(batch) + '-normalized_64_test.npy')
    X_test = X_test.reshape(X_test.shape[0], 64, 64, 1)
    X_test = X_test.astype('float32')
    X_test /= 255
    pred = model.predict_classes(X_test, batch_size = 100, verbose=1)
    for elem in pred:
        good = False
        output.write(str(c) + ',')
        for _c in class_values:
            if class_values[_c] == elem:
                if good:
                    print("ERROR")
                output.write(str(_c) + '\n')
                good = True
        c += 1
    del X_test
    del pred
output.close()

1428/1428 [==============================] - 10s    


Еще ячейка для дообучения:

In [20]:
for batch in range(1, 34):
    X_train = numpy.load(str(batch) + '-normalized_64_pictures.npy')
    Y_train = numpy.load(str(batch) + '-normalized_64_answers.npy')

    for i in range(len(Y_train)):
        Y_train[i] = class_values[Y_train[i]] 
    Y_train = np_utils.to_categorical(Y_train, len(class_values))

    X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
    X_train = X_train.astype('float32')
    X_train /= 255

    history = model.fit(X_train, Y_train,
                        batch_size=128, nb_epoch=1,
                        verbose=0, validation_data=(X_train, Y_train))
    score = model.evaluate(X_train, Y_train, verbose=0)
    print(batch, score)
    del X_train
    del Y_train

1 [0.02021203575283289, 0.99980000000000002]
2 [0.01615585423335433, 0.99939999999999996]
3 [0.023033753728121519, 0.99739999999999995]
4 [0.027365677703544498, 0.996]
5 [0.022404221490211786, 0.997]
6 [0.017041471758764238, 0.99760000000000004]
7 [0.020513035023957491, 0.99760000000000004]
8 [0.017003282930515708, 0.99780000000000002]
9 [0.014174783427268267, 0.99860000000000004]
10 [0.010263341222144663, 0.99960000000000004]
11 [0.010880385356023908, 0.99839999999999995]
12 [0.019346642546728254, 0.99780000000000002]
13 [0.024744336139410735, 0.998]
14 [0.017890634779632091, 0.99780000000000002]
15 [0.022488429614156483, 0.99639999999999995]
16 [0.026737224546819927, 0.996]
17 [0.025343405513465405, 0.99660000000000004]
18 [0.031413016498833893, 0.99760000000000004]
19 [0.042522468450106682, 0.99619999999999997]
20 [0.023028140731155871, 0.998]
21 [0.024072759718447925, 0.997]
22 [0.048793598934262994, 0.99280000000000002]
23 [0.032391246946528555, 0.99580000000000002]
24 [0.02523250

Ячейка для сравнения ответов нескольких нейронок и выбор лучшего ответа:

In [6]:
file1 = open('pred_64_1.txt', 'r')
file2 = open('pred_64_4.txt', 'r')
file3 = open('pred_64_common_8.txt', 'r')
file_ans = open('pred_64_common_11.txt', 'w')
s1 = file1.readline()
s2 = file2.readline()
s3 = file3.readline()
file_ans.write(s1)
s1 = file1.readline()
s2 = file2.readline()
s3 = file3.readline()
c1 = 0
c2 = 0
while len(s1) > 0:
    s1 = s1.split(',')
    s2 = s2.split(',')
    s3 = s3.split(',')
    file_ans.write(s1[0] + ',')
    ans1 = int(s1[1])
    ans2 = int(s2[1])
    ans3 = int(s3[1])
    if not(ans1 == ans2 == ans3):
        c1 += 1
    if ans1 == ans2:
        file_ans.write(str(ans1))
    elif ans1 == ans3:
        file_ans.write(str(ans1))
    elif ans2 == ans3:
        file_ans.write(str(ans2))
    else:
        file_ans.write(str(ans3))
        print(s1[0], ans1, ans2, ans3)
        c2 += 1
    file_ans.write('\n')
    s1 = file1.readline()
    s2 = file2.readline()
    s3 = file3.readline()
print(c1, c2)
file_ans.close()
file1.close()
file2.close()
file3.close()

357 65223 62146 64696
689 64691 64972 64708
763 64470 64464 64468
1108 65217 62416 62654
1353 62385 62897 64944
1427 62412 63415 62668
1524 63444 62665 62898
1557 62656 62402 65204
1740 63165 62667 63683
1777 62664 64451 63430
1795 62398 64711 62666
1867 63688 63949 63437
1939 62674 62906 62419
1942 62914 63180 63187
2035 62158 63154 62143
2361 62666 64192 63424
2379 64190 62907 64961
2394 62147 63163 63409
2408 62143 62678 62404
2480 62401 62657 63177
2540 64443 62896 63189
2706 63446 64961 63161
2709 62922 64969 64955
2713 62393 63419 62664
2716 63165 63160 62903
2740 65220 64439 63177
2815 63958 64452 62640
2837 63951 64965 63430
3087 63152 62675 64440
4080 64192 63680 62899
4132 63927 63169 63671
4182 63675 64436 64944
4327 62669 64710 63954
4411 63958 62678 63437
4562 63431 63178 64713
4638 63413 62405 63427
4641 63691 64446 63673
4673 62160 62662 64439
4896 63924 62896 65233
4985 64177 64433 63671
4987 64447 62658 62918
5008 62651 63941 62648
5021 64973 62656 64692
5044 63938 644